# 分布式TensorFlow简介

## 起个Cluster

看task.py文件，了解ps服务器和worker服务器。

进入容器后启动三个进程模仿三个服务器（这里纯粹为了快速演示）

```
docker exec -it tflab bash
    python task.py --job_name=ps --task_index=0
docker exec -it tflab bash
    python task.py --job_name=worker --task_index=0
docker exec -it tflab bash
    python task.py --job_name=worker --task_index=1
```

查看日志文件看看输出是什么

## 图并行

### 定义个client
最简单的model

In [ ]:
import tensorflow as tf
import numpy as np

cluster_spec = {
    "ps": ["localhost:50001"],
    "worker": ["localhost:60001", "localhost:60002"]}
graph = tf.Graph()
with graph.as_default():
    with tf.device(tf.train.replica_device_setter(cluster=cluster_spec)):      
        i = tf.placeholder(tf.float32, shape=(None, 1), name="input")
        a = tf.Variable(tf.constant(10.0, shape=[1]), name="a")
        b = tf.Variable(tf.constant(20.0, shape=[1]), name="b")
        node1 = tf.add(i, a, name="node1")
        node2 = tf.add(i, b, name="node2")
        node3 = tf.add(node1, node2, name="node3")

with tf.Session("grpc://localhost:60001",
                graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    data = np.array([2]).reshape(-1, 1)
    output = sess.run(node3, feed_dict={i: data})
    print(output)

### 思考上面的程序是如何在各个服务器上运行的

修改使得图并行

In [ ]:
graph2 = tf.Graph()
with graph2.as_default():
    i = tf.placeholder(tf.float32, shape=(None, 1))
    with tf.device("/job:ps/task:0"):
        a = tf.Variable(tf.constant(100.0, shape=[1]), name="a")
        b = tf.Variable(tf.constant(200.0, shape=[1]), name="b")
    with tf.device("/job:worker/task:0"):
        node1 = tf.add(i, a, name="node1")
    with tf.device("/job:worker/task:1"):
        node2 = tf.add(i, b, name="node2")
    # Think if we don't specify device, who will handle it?
    # Tip: config=tf.ConfigProto(log_device_placement=True)
    node3 = tf.add(node1, node2, name="node3")

with tf.Session("grpc://localhost:60002", graph=graph2) as sess:
    sess.run(tf.global_variables_initializer())
    data = np.array([2]).reshape(-1, 1)
    node3out = sess.run(node3,
                       feed_dict={i: data})
    print(node3out)

> 请自行设计并完成两种数据并行的实验